## Final Project Submission

* Student names:

Jeremiah Waiguru

Olive Muloma

Troye Gilbert

Josephine Maro


* Student pace: full time
* Scheduled project review date/time: N/A
* Instructor name: Maryann Mwikali
* Blog post URL: N/A


### Data
We have been provided with a dataset with house sale prices in King County, Washington State, USA from 2014 to 2015 to use for this project.

A dataset has been provided and can be found in the kc_house_data.csv file in this repository.

The column names and descriptions as provided can be found in the column_names.md file in this repository.



In [3]:
# import the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as sfm
import statsmodels.api as sm
import scipy.stats as stats
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

### loading dataset

In [13]:
# loading the data and previewing the dataframe
df = pd.read_csv('data/kc_house_data.csv', index_col=0)
df.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
id,,,,,,,,,,,,,,,,,,,,
7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,NONE,Average,7 Average,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,NO,NONE,Average,7 Average,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,NO,NONE,Average,6 Low Average,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,NO,NONE,Very Good,7 Average,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,NO,NONE,Average,8 Good,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


## Data Preparation
In this section, we shall be preparing the data for further processing and modelling

### Investigate data types

In [17]:
# shape of our data
df.shape

(21597, 20)

In [16]:
# Describing the data
df.describe()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,sqft_above,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.159700e+04,21597.000000,21597.000000,21597.000000,2.159700e+04,21597.000000,21597.000000,21597.000000,17755.000000,21597.000000,21597.000000,21597.000000,21597.000000,21597.000000
mean,5.402966e+05,3.373200,2.115826,2080.321850,1.509941e+04,1.494096,1788.596842,1970.999676,83.636778,98077.951845,47.560093,-122.213982,1986.620318,12758.283512
std,3.673681e+05,0.926299,0.768984,918.106125,4.141264e+04,0.539683,827.759761,29.375234,399.946414,53.513072,0.138552,0.140724,685.230472,27274.441950
min,7.800000e+04,1.000000,0.500000,370.000000,5.200000e+02,1.000000,370.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000
25%,3.220000e+05,3.000000,1.750000,1430.000000,5.040000e+03,1.000000,1190.000000,1951.000000,0.000000,98033.000000,47.471100,-122.328000,1490.000000,5100.000000
50%,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,1560.000000,1975.000000,0.000000,98065.000000,47.571800,-122.231000,1840.000000,7620.000000
75%,6.450000e+05,4.000000,2.500000,2550.000000,1.068500e+04,2.000000,2210.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000
max,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,9410.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


In [15]:
# summary of the data
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21597 entries, 7129300520 to 1523300157
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           21597 non-null  object 
 1   price          21597 non-null  float64
 2   bedrooms       21597 non-null  int64  
 3   bathrooms      21597 non-null  float64
 4   sqft_living    21597 non-null  int64  
 5   sqft_lot       21597 non-null  int64  
 6   floors         21597 non-null  float64
 7   waterfront     19221 non-null  object 
 8   view           21534 non-null  object 
 9   condition      21597 non-null  object 
 10  grade          21597 non-null  object 
 11  sqft_above     21597 non-null  int64  
 12  sqft_basement  21597 non-null  object 
 13  yr_built       21597 non-null  int64  
 14  yr_renovated   17755 non-null  float64
 15  zipcode        21597 non-null  int64  
 16  lat            21597 non-null  float64
 17  long           21597 non-null  float64
 1

The dataset contains 21,597 entries and 20 columns

Some columns like 'waterfront', 'view', 'yr_renovated' have missing values


### loading the column.md dataset

In [28]:
with open('data/column_names.md', 'r') as file:
    md_lines = file.readlines()

df1 = pd.DataFrame({'Text': md_lines})

pd.set_option('display.max_colwidth',None)

# df1

#### cleaning the column_md dataset

In [30]:
df1[['Column name', 'Descriptions']] = df1['Text'].str.split('-', n=1,expand=True)

#### drop the original 'text' column

In [31]:
df1.drop(columns=['Text'], inplace=True)

#### remove rows where 'descriptions' columns contains 'None'

In [32]:
df1 = df1[df1['Descriptions'].notna()]
df1

,Column name,Descriptions
1,* `id`,Unique identifier for a house\n
2,* `date`,Date house was sold\n
3,* `price`,Sale price (prediction target)\n
4,* `bedrooms`,Number of bedrooms\n
5,* `bathrooms`,Number of bathrooms\n
6,* `sqft_living`,Square footage of living space in the home\n
7,* `sqft_lot`,Square footage of the lot\n
8,* `floors`,Number of floors (levels) in house\n
9,* `waterfront`,Whether the house is on a waterfront\n
11,* `view`,Quality of view from house\n


In [33]:
# checking column names
df.columns

Index(['date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [9]:
# function to check null values
def check_null(df):
    return df.isna().sum()

# checking for null values in the data
check_null(df)

id                  0
date                0
price               0
bedrooms            0
bathrooms           0
sqft_living         0
sqft_lot            0
floors              0
waterfront       2376
view               63
condition           0
grade               0
sqft_above          0
sqft_basement       0
yr_built            0
yr_renovated     3842
zipcode             0
lat                 0
long                0
sqft_living15       0
sqft_lot15          0
dtype: int64

There are missing values in three columns.

Depending on the ratio of missing values, we will decide on what approach to take in dealing with them

## Dealing with missing values


In [10]:
# function to calculate percentage of null values
def miss_percent(df,col):
    miss = ((df[col].isna().sum()) / len(df[col])) * 100
    return print(f'There is {miss} percent of values missing in {col}.')

# checking percentage of missing values 
miss_percent(df,'waterfront')
miss_percent(df, 'view')
miss_percent(df, 'yr_renovated')

There is 11.00152798999861 percent of values missing in waterfront.
There is 0.29170718155299347 percent of values missing in view.
There is 17.78950780200954 percent of values missing in yr_renovated.


The threshold on how to deal with missing values commonly used is 50% and also depends on the specific column. The percentages of missing values are very low for the specific columns so we can replace.

Checking the year renovated column we may assume the missing value is because the house was never renovated, maybe the house did not have a view or a waterfront also for the other two columns hence we can Fill them with zeros.

Since the missing values in the 3 columns are categorical and are a small percentage of the columns, replacing them with mode won`t skew the data nor give false conclusions

In [21]:
def miss_percent(df, col, fill_value=None):
    miss = ((df[col].isna().sum()) / len(df[col])) * 100
    if fill_value is not None:
        df[col].fillna(fill_value, inplace=True)
    return miss

# checking percentage of missing values and filling missing values with the mode
fill_values = {'waterfront': df['waterfront'].mode()[0], 
               'view': df['view'].mode()[0], 
               'yr_renovated': df['yr_renovated'].mode()[0]}

for col in fill_values:
    missing_percent = miss_percent(df, col, fill_value=fill_values[col])
    print(f'There is {missing_percent:.2f}% of values missing in {col}.')

There is 0.00% of values missing in waterfront.
There is 0.00% of values missing in view.
There is 0.00% of values missing in yr_renovated.


### check duplicates

In [23]:
duplicates = df.duplicated().sum()
duplicates

0